In [ ]:
# By Huachen Zhang from T06, IOP, CAS

In [1]:
using LinearAlgebra
include("ncon.jl") # The Network Contraction program by Glen Evenbly at Georgia Institute of Technology. If you want to run my program, please place this file in the working directory!

ncon_check_inputs

In [4]:
# 1. The free energy density of an AFM Ising model defined on a C60 molecule. The magnitude of coupling constant and temperature are both set to unity.

eye3 = zeros(2, 2, 2); # The "unity tensor" of order 3 (the generalisation of identity matrix) for "star" contraction in the initialisation of the tensor network.
for i in 1:2
    for j in 1:2
        for k in 1:2
            if i == j && j == k
                eye3[i, j, k] = 1;
            end
        end
    end
end

function GetTensor(beta) # Initialisation of the tensor network; beta is the inverse temperature.
    W = [exp(-beta) exp(beta); exp(beta) exp(-beta)]; # Boltzmann weight on the bonds, AFM Ising
    F = eigen(W);
    M = (sqrt(Diagonal(F.values + [0.0im, 0])))*(F.vectors);
    T3 = ncon(Any[eye3, M, M, M], Any[[1, 2, 3], [-1, 1], [-2, 2], [-3, 3]]);
    return T3
end

T3 = GetTensor(1); # beta = 1
T5 = ncon(Any[T3, T3, T3, T3, T3], Any[[-1, 1, 2], [-2, 3, 1], [-3, 4, 3], [-4, 5, 4], [-5, 2, 5]]); # The contraction of the five order-3 tensors around one pentagon.
T10 = ncon(Any[T5, T5, T5, T5, T5, T5], Any[[1, 2, 3, 4, 5], [1, 6, -1, -2, 7], [2, 7, -3, -4, 8], [3, 8, -5, -6, 9], [4, 9, -7, -8, 10], [5, 10, -9, -10, 6]]); # The order-10 tensor from the contraction of six order-5 tensors.
Z = real(ncon(Any[T10, T10], Any[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [10, 1, 2, 3, 4, 5, 6, 7, 8, 9]])); # The exact contraction of all tensors gives the partition function of the model.
FEnergy_density = (-log(Z))/60;
println(FEnergy_density)

-1.3073684577607936


In [5]:
# 2. The degeneracy of the (classical) ground state.
# I solve this problem by mapping it to the contraction of a tensor network exactly.
# Another way of (approximately) calculating the degeneracy of the ground state is given by my collaborator Song Sun from T03, IOP, CAS.
# Yet another way is the automatic differentiation of the free energy with respect to temperature. We do not have enough time to do this.

# There are 90 bonds in total. In the ground states, the spin configuration around each pentagon has 10 possibilities (since it is frustrated);
# and there is an additional restriction that the 30 bonds solely within hexagonals must have opposite spins on their ends.

T = zeros(2, 2, 2, 2, 2);
# The non-zero elements of the order-5 tensor correspond to the 10 possible spin configurations around one pentagon.
for i in 1:2
    for j in 1:2
        for k in 1:2
            for l in 1:2
                for m in 1:2
                    if (([i j k l m] == [2 1 2 1 1])||([i j k l m] == [2 1 1 2 1])||([i j k l m] == [1 2 1 1 2])||([i j k l m] == [1 1 2 1 2])||([i j k l m] == [1 2 1 2 1])||([i j k l m] == [1 2 1 2 2])||([i j k l m] == [1 2 2 1 2])||([i j k l m] == [2 1 2 2 1])||([i j k l m] == [2 2 1 2 1])||([i j k l m] == [2 1 2 1 2]))
                        T[i, j, k, l, m] = 1;
                    end
                end
            end
        end
    end
end

X = [0 1; 1 0]; # The spin-flip matrix used in the imposition of the additional restriction mentioned above.

# The contraction of the "upper half" of the system.
Tu = ncon(Any[T, X, X, X, X, X, T, X, X, X, X, T, X, X, X, T, X, X, X, T, X, X, X, T, X, X], Any[[1, 2, 3, 4, 5], [1, 6], [2, 7], [3, 8], [4, 9], [5, 10], [6, 11, 21, 22, 12], [20, 11], [21, -1], [22, -2], [12, 13], [7, 13, 23, 24, 14], [23, -3], [24, -4], [14, 15], [8, 15, 25, 26, 16], [25, -5], [26, -6], [16, 17], [9, 17, 27, 28, 18], [27, -7], [28, -8], [18, 19], [10, 19, 29, 30, 20], [29, -9], [30, -10]]);
# The contraction of the "lower half" of the system.
Td = ncon(Any[T, X, X, X, X, X, T, X, X, T, X, T, X, T, X, T], Any[[1, 2, 3, 4, 5], [1, 6], [2, 7], [3, 8], [4, 9], [5, 10], [6, 11, -1, -2, 12], [20, 11], [12, 13], [7, 13, -3, -4, 14], [14, 15], [8, 15, -5, -6, 16], [16, 17], [9, 17, -7, -8, 18], [18, 19], [10, 19, -9, -10, 20]]);
# Finally,
Degeneracy = ncon(Any[Tu, Td], Any[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [10, 1, 2, 3, 4, 5, 6, 7, 8, 9]]);
println(Degeneracy)

16000.0
